In [146]:
# configuración para notebook con instalación LOCAL
FUENTES_DIR  = '../Fuentes'     # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR    = '../Datos/'      # carpeta donde se encuentran los datasets
MODELOS_DIR  = '../Modelos/'    # carpeta donde se encuentran los modelos
ARCHIVOs_DIR = '../Archivos/'   # carpeta recuperar o copiar archivos

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

In [147]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from matplotlib import pylab as plt

from grafica import *
from ClassNeuronaGral import NeuronaGradiente

In [148]:
import chardet


nombre_archivo = DATOS_DIR + 'Vinos.csv' 

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv(nombre_archivo, sep=';')

# visualiza el dataframe

df

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [149]:
print(df["Class"].unique())

[1 2 3]


In [150]:
dummies = pd.get_dummies(df['Class'], prefix='Class').astype(int)

df = pd.concat([df, dummies], axis=1)
df = df.drop('Class', axis=1)
df

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline,Class_1,Class_2,Class_3
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1,0,0
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1,0,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1,0,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1,0,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,0,0,1
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,0,0,1
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,0,0,1
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,0,0,1


In [151]:
#Division entre X e Y
Y = df["Class_1"]
X = df.drop(columns="Class_1")

X = np.array(X)
Y = np.array(Y)

In [152]:
#Normalizmos los datos
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

In [153]:
from sklearn.model_selection import train_test_split

def ej_diez(funcion, costo, vel_apr):
    CANT_EJECUCIONES = 30
    PORCENTAJES = [0.5, 0.6, 0.7, 0.8, 0.9]
    MAX_ITE = 400
    VELOCIDAD_APR = vel_apr
    FUNC = funcion
    COSTO = costo
    res = []

    for i in PORCENTAJES:
        ejemplos_correctos_train = []
        ejemplos_correctos_test = []
        cant_iteraciones = []

        for p in range (CANT_EJECUCIONES):
            #Primero dividimos los datos en entrenamiento y testeo:
            X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=i, random_state=None)
            #model
            neurona = NeuronaGradiente(alpha=VELOCIDAD_APR, n_iter=MAX_ITE, FUN=FUNC, COSTO=COSTO, draw=0)
            #training
            neurona.fit(X_train, y_train)
            #predictions
            y_pred_train = neurona.predict(X_train)
            y_pred_test = neurona.predict(X_test)
            #sum of the correctly classified
            correct_train = np.sum(y_pred_train == y_train)
            correct_test = np.sum(y_pred_test == y_test)
            #save the info
            ejemplos_correctos_train.append(correct_train)
            ejemplos_correctos_test.append(correct_test)
            cant_iteraciones.append(len(neurona.errors_))

        prom_train = np.mean(ejemplos_correctos_train)
        prom_test = np.mean(ejemplos_correctos_test)
        prom_iter = np.mean(cant_iteraciones)
        desv_iter = np.std(cant_iteraciones)

        res.append({
            'Porcentaje': i,
            'Tasa' : VELOCIDAD_APR,
            'Promedio Correctos Train': prom_train,
            'Promedio Correctos Test': prom_test,
            'Promedio Iteraciones': prom_iter,
            'Desviación Iteraciones': desv_iter
        })

    for r in res:
        print(f"Porcentaje: {r['Porcentaje']}, Tasa: {r['Tasa']}")
        print(f"  Promedio Correctos Train: {r['Promedio Correctos Train']}")
        print(f"  Promedio Correctos Test: {r['Promedio Correctos Test']}")
        print(f"  Promedio Iteraciones: {r['Promedio Iteraciones']} +/- {r['Desviación Iteraciones']}")
        print()

In [154]:
ej_diez("sigmoid", "ECM", 0.1)
ej_diez("sigmoid", "ECM", 0.2)
ej_diez("sigmoid", "ECM", 0.3)

ej_diez("sigmoid", "EC_binaria", 0.1)
ej_diez("sigmoid", "EC_binaria", 0.2)
ej_diez("sigmoid", "EC_binaria", 0.3)

ej_diez("tanh", "ECM", 0.1)
ej_diez("tanh", "ECM", 0.2)
ej_diez("tanh", "ECM", 0.3)


Porcentaje: 0.5, Tasa: 0.1
  Promedio Correctos Train: 89.0
  Promedio Correctos Test: 88.66666666666667
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.6, Tasa: 0.1
  Promedio Correctos Train: 106.0
  Promedio Correctos Test: 71.9
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.7, Tasa: 0.1
  Promedio Correctos Train: 124.0
  Promedio Correctos Test: 54.0
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.8, Tasa: 0.1
  Promedio Correctos Train: 142.0
  Promedio Correctos Test: 36.0
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.9, Tasa: 0.1
  Promedio Correctos Train: 160.0
  Promedio Correctos Test: 18.0
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.5, Tasa: 0.2
  Promedio Correctos Train: 89.0
  Promedio Correctos Test: 88.8
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.6, Tasa: 0.2
  Promedio Correctos Train: 106.0
  Promedio Correctos Test: 71.96666666666667
  Promedio Iteraciones: 400.0 +/- 0.0

Porcentaje: 0.7, Tasa: 0.2
  Promedio Correctos Trai